In [2]:
import os
import sys
import pandas as pd
import argparse
from datetime import datetime, timedelta
import pathlib

# 新增：將上一層目錄加入 sys.path
parent_path = pathlib.Path().absolute().parent
sys.path.insert(0, str(parent_path))
# Setup Django environment
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'website_configs.settings')
import django
django.setup()
# 重要：設定環境變數以允許在 Jupyter 的異步環境中執行同步操作
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Now we can import Django models
from app_user_keyword_db.models import NewsData

In [3]:
from django.db.models import Q, Max, F
from collections import Counter

In [5]:
# Searching keywords from "content" column
# This function now uses database queries instead of pandas
def filter_database_fullText(user_keywords, cond, cate, weeks):
    # Get the latest date in the database
    latest_date = NewsData.objects.aggregate(max_date=Max('date'))['max_date']
    
    # Calculate start date
    start_date = latest_date - timedelta(weeks=weeks)
    
    # Base query - filter by date range
    queryset = NewsData.objects.filter(date__gte=start_date, date__lte=latest_date)
    
    # Filter by category if not "全部"
    if cate != "全部":
        queryset = queryset.filter(category=cate)
    
    # Filter by keywords based on condition (AND or OR)
    if cond == 'and':
        # For AND condition, we need all keywords to be present
        for kw in user_keywords:
            queryset = queryset.filter(content__contains=kw)
    elif cond == 'or':
        # For OR condition, any keyword can be present
        q_objects = Q()
        for kw in user_keywords:
            q_objects |= Q(content__contains=kw)
        queryset = queryset.filter(q_objects)
    
    return queryset

In [8]:

user_keywords = ['川普', '俄羅斯']  # Example keywords
cond = 'or'  # Example condition (and/or), 
cate = '全部'  # Example category (or "全部" for all categories)
weeks = 4  # Example weeks
queryset = filter_database_fullText(user_keywords, cond, cate, weeks)

In [9]:
for news in queryset:
    print(news.title)

川普上任兩個月 各項關稅概況一覽
法院文件揭密 司法部高層施壓撤銷亞當斯案
餐廳重設戶外餐區 華埠小商家因2原因對生意仍信心不足
躲不過川普「汽車關稅」！新車價格喊漲　驚人數字曝光
收盤／危險啊！權值股倒成一片　大盤跌破2萬2！
股市一直跌誰最慘？「1類人」恐成海嘯第一排　專家曝3方法降低風險


In [10]:
# Limit to k results and get specific fields
news_items = queryset.values('category', 'title', 'link', 'photo_link')[:3]

In [11]:
news_items

<QuerySet [{'category': '國際', 'title': '川普上任兩個月 各項關稅概況一覽', 'link': 'https://tw.news.yahoo.com/%E5%B7%9D%E6%99%AE%E4%B8%8A%E4%BB%BB%E5%85%A9%E5%80%8B%E6%9C%88-%E5%90%84%E9%A0%85%E9%97%9C%E7%A8%85%E6%A6%82%E6%B3%81-%E8%A6%BD-070602268.html', 'photo_link': None}, {'category': '國際', 'title': '法院文件揭密 司法部高層施壓撤銷亞當斯案', 'link': 'https://tw.news.yahoo.com/%E6%B3%95%E9%99%A2%E6%96%87%E4%BB%B6%E6%8F%AD%E5%AF%86-%E5%8F%B8%E6%B3%95%E9%83%A8%E9%AB%98%E5%B1%A4%E6%96%BD%E5%A3%93%E6%92%A4%E9%8A%B7%E4%BA%9E%E7%95%B6%E6%96%AF%E6%A1%88-063116606.html', 'photo_link': None}, {'category': '國際', 'title': '餐廳重設戶外餐區 華埠小商家因2原因對生意仍信心不足', 'link': 'https://tw.news.yahoo.com/%E9%A4%90%E5%BB%B3%E9%87%8D%E8%A8%AD%E6%88%B6%E5%A4%96%E9%A4%90%E5%8D%80-%E8%8F%AF%E5%9F%A0%E5%B0%8F%E5%95%86%E5%AE%B6%E5%9B%A02%E5%8E%9F%E5%9B%A0%E5%B0%8D%E7%94%9F%E6%84%8F%E4%BB%8D%E4%BF%A1%E5%BF%83%E4%B8%8D%E8%B6%B3-062900330.html', 'photo_link': 'https://s.yimg.com/uu/api/res/1.2/sQvI4HVzFa0tBIg6cYZ94A--~B/Zmk9c3RyaW07aD0yMDA7cT04MDt3PTIwMDth

In [12]:
dates = list(queryset.values_list('date', flat=True))

In [13]:
NewsData.objects.filter(category ='國際').values('category', 'title','date')

<QuerySet [{'category': '國際', 'title': '特朗普宣布美國對進口汽車徵收25%關稅', 'date': datetime.date(2025, 3, 27)}, {'category': '國際', 'title': '香港長和賣港口 中國商務部專家批「在商不言商」', 'date': datetime.date(2025, 3, 27)}, {'category': '國際', 'title': '川普上任兩個月 各項關稅概況一覽', 'date': datetime.date(2025, 3, 27)}, {'category': '國際', 'title': '中國男大生坐「共享單車兒童椅」\u3000卡死出不來！急求助消防員', 'date': datetime.date(2025, 3, 27)}, {'category': '國際', 'title': '陸2旅客身藏「28隻活體龜」入境…遭海關查獲\u3000離譜走私手法曝', 'date': datetime.date(2025, 3, 27)}, {'category': '國際', 'title': '大陸玩具商「泡泡瑪特」2024營收翻倍！\u3000發豪語「邁向千億營收」', 'date': datetime.date(2025, 3, 27)}, {'category': '國際', 'title': '法院文件揭密 司法部高層施壓撤銷亞當斯案', 'date': datetime.date(2025, 3, 27)}, {'category': '國際', 'title': '白宮Signal門：《大西洋月刊》披露群聊全文，美國安團隊洩密事件六大疑點', 'date': datetime.date(2025, 3, 27)}, {'category': '國際', 'title': '華人居民控告亞當斯 「同意之城」掠奪社區土地', 'date': datetime.date(2025, 3, 27)}, {'category': '國際', 'title': '餐廳重設戶外餐區 華埠小商家因2原因對生意仍信心不足', 'date': datetime.date(2025, 3, 27)}]>